Overload the Irrational print statement to avoid printing the value of pi with symbol

In [1]:
Base.show(io::IO, x::Irrational{sym}) where {sym} = print(io, "$sym")

In [104]:
"""
    generate_gate(gate_name)

Generates an OpenQASM-style gate structure string.
The internals of {} can be replace with the appropriate
values.

# Examples
```julia-repl
julia> generate_gate("Q")
\"gate Q {}\"
```
"""
function generate_gate(gate_name::String)
   return "gate " * gate_name * " {}" 
end

generate_gate

In [4]:
qasm_map = Dict{String, String}()

Dict{String,String} with 0 entries

In [5]:
qasm_map["U"] = "U(theta,phi,lambda) qReg[q_idx];"
qasm_map["CX"] = "CX c,t;"
qasm_map["T"] = "t qReg[q_idx];"
qasm_map["TDG"] = "tdg qReg[q_idx];"
qasm_map["S"] = "s qReg[q_idx];"
qasm_map["SDG"] = "sdg qReg[q_idx];";

In [6]:
struct BlochAngles
    θ::Real
    ϕ::Real
    λ::Real
end

In [7]:
function piConvert(value::Real)
    return typeof(value)<:Irrational{:π} ? "pi" : value
end

piConvert (generic function with 1 method)

In [88]:
function applyGateU(angles::BlochAngles, qReg::String, qIdx::Union{Int, Nothing}=nothing)
   
    result = replace(qasm_map["U"], "theta"=>piConvert(angles.θ))
    result = replace(result, "phi"=>piConvert(angles.ϕ))
    result = replace(result, "lambda"=>piConvert(angles.λ))
    result = replace(result, "qReg"=>qReg)
    
    #If no index given, apply same ops across entire register
    if qIdx == nothing 
        result = replace(result, "[q_idx]"=>"")
    else # Otherwise apply op to given qubit index
        result = replace(result, "q_idx"=>qIdx)
    end
    return result
end

#Pauli and Hadamard
function applyGateX(qReg::String, qIdx::Union{Int, Nothing}=nothing)
    return applyGateU( BlochAngles(pi, 0, pi), qReg, qIdx)
end
function applyGateY(qReg::String, qIdx::Union{Int, Nothing}=nothing)
    return applyGateU( BlochAngles(pi, pi/2, pi/2), qReg, qIdx)
end
function applyGateZ(qReg::String, qIdx::Union{Int, Nothing}=nothing)
    return applyGateU( BlochAngles(0, 0, pi), qReg, qIdx)
end
function applyGateH(qReg::String, qIdx::Union{Int, Nothing}=nothing)
    return applyGateU( BlochAngles(pi/2, 0, pi), qReg, qIdx)
end

function applyGateT(qReg::String, qIdx::Union{Int, Nothing}=nothing, isAdjoint::Union{Bool, Nothing}=nothing)
    #If no index given, apply same ops across entire register
    if isAdjoint == nothing
        result = qasm_map["T"]
    else
        result = qasm_map["TDG"]
    end        
    
    result = replace(result, "qReg"=>qReg)
    
    if qIdx == nothing
        result = replace(result, "[q_idx]"=>"")
    else # Otherwise apply op to given qubit index
        result = replace(result, "q_idx"=>qIdx)
    end
    return result
end

function applyGateS(qReg::String, qIdx::Union{Int, Nothing}=nothing, isAdjoint::Union{Bool, Nothing}=nothing)
    #If no index given, apply same ops across entire register
    if isAdjoint == nothing
        result = qasm_map["S"]
    else
        result = qasm_map["SDG"]
    end        
    
    result = replace(result, "qReg"=>qReg)
    
    if qIdx == nothing
        result = replace(result, "[q_idx]"=>"")
    else # Otherwise apply op to given qubit index
        result = replace(result, "q_idx"=>qIdx)
    end
    return result
end

#Rotation
function applyGateRX(qReg::String, θ::Real, qIdx::Union{Int, Nothing}=nothing)
    return applyGateU( BlochAngles(θ,-pi/2,pi/2), qReg, qIdx)
end
function applyGateRY(qReg::String, θ::Real, qIdx::Union{Int, Nothing}=nothing)
    return applyGateU( BlochAngles(θ,0,0), qReg, qIdx)
end
function applyGateRZ(qReg::String, ϕ::Real, qIdx::Union{Int, Nothing}=nothing)
    return applyGateU( BlochAngles(0, 0, ϕ), qReg, qIdx)
end

#2 qubit
function applyGateCX(qReg::String, qCtrlIdx::Int, qTgtIdx::Int)
    result = replace(qasm_map["CX"], "c"=>"$qReg[$qCtrlIdx]")
    result = replace(result, "t"=>"$qReg[$qTgtIdx]")
    return result
end

function applyGateCY(qReg::String, qCtrlIdx::Int, qTgtIdx::Int)
    result =  applyGateU( BlochAngles(0, 0, -pi/2), qReg, qTgtIdx);
    result *= applyGateCX(qReg, qCtrlIdx, qTgtIdx)
    result *= applyGateU( BlochAngles(0, 0, pi/2), qReg, qTgtIdx);
    return result
end

function applyGateCZ(qReg::String, qCtrlIdx::Int, qTgtIdx::Int)
    result =  applyGateH(qReg, qTgtIdx)
    result *= applyGateCX(qReg, qCtrlIdx, qTgtIdx)
    result *= applyGateH(qReg, qTgtIdx)
    return result 
end

function applyGateCCX(qReg::String, qCtrlIdx1::Int, qCtrlIdx2::Int, qTgtIdx::Int)
    result =  applyGateH(qReg, qTgtIdx)
    result *= applyGateCX(qReg, qCtrlIdx2, qTgtIdx)
    result *= applyGateT(qReg, qTgtIdx, true)
    result *= applyGateCX(qReg, qCtrlIdx1, qTgtIdx)
    result *= applyGateT(qReg, qTgtIdx, false)
    result *= applyGateCX(qReg, qCtrlIdx2, qTgtIdx)
    result *= applyGateT(qReg, qTgtIdx, true)
    result *= applyGateCX(qReg, qCtrlIdx1, qTgtIdx)
    result *= applyGateT(qReg, qCtrlIdx2, false)
    result *= applyGateT(qReg, qTgtIdx, false)
    result *= applyGateH(qReg, qTgtIdx)
    result *= applyGateCX(qReg, qCtrlIdx1, qCtrlIdx2)
    result *= applyGateT(qReg, qCtrlIdx1, false)
    result *= applyGateT(qReg, qCtrlIdx2, true)
    result *= applyGateCX(qReg, qCtrlIdx1, qCtrlIdx2)
    return result 
end

function applyGateCPhase(qReg::String, λ::Real, qCtrlIdx::Int, qTgtIdx::Int)
    # [[1,0],[0, exp(iλ)]]
    result =  applyGateU( BlochAngles(0, 0, λ/2), qReg, qCtrlIdx);
    result *= applyGateCX(qReg, qCtrlIdx, qTgtIdx);
    result *= applyGateU( BlochAngles(0, 0, -λ/2), qReg, qTgtIdx);
    result *= applyGateCX(qReg, qCtrlIdx, qTgtIdx);
    result *= applyGateU( BlochAngles(0, 0, λ/2), qReg, qTgtIdx);
end

function applySwap(qReg::String, q1::Int, q2::Int)
    result =  applyGateCX(qReg, q1, q2)
    result *= applyGateCX(qReg, q2, q1)
    result *= applyGateCX(qReg, q1, q2)
    return result
end

applySwap (generic function with 1 method)

In [103]:
IBM_INC_HEADER="""
gate u3(theta,phi,lambda) q { U(theta,phi,lambda) q; }
gate u2(phi,lambda) q { U(pi/2,phi,lambda) q; }
gate u1(lambda) q { U(0,0,lambda) q; }
gate cx c,t { CX c,t; }
gate id a { U(0,0,0) a; }
gate x a { u3(pi,0,pi) a; }
gate y a { u3(pi,pi/2,pi/2) a; }
gate z a { u1(pi) a; }
gate h a { u2(0,pi) a; }
gate rx(theta) a { u3(theta,-pi/2,pi/2) a; }
gate ry(theta) a { u3(theta,0,0) a; }
gate rz(phi) a { u1(phi) a; }
gate cz a,b { h b; cx a,b; h b; }
gate cy a,b { sdg b; cx a,b; s b; }
gate ch a,b {h b; sdg b;cx a,b;h b; t b;cx a,b;t b; h b; s b; x b; s a;}
gate ccx a,b,c{h c;cx b,c; tdg c;cx a,c; t c;cx b,c; tdg c;cx a,c; t b; t c; h c;cx a,b; t a; tdg b;cx a,b;}
gate cu1(lambda) a,b{u1(lambda/2) a;cx a,b;u1(-lambda/2) b;cx a,b;u1(lambda/2) b;}
gate cu3(theta,phi,lambda) c, t{u1((lambda-phi)/2) t;cx c,t;u3(-theta/2,0,-(phi+lambda)/2) t;cx c,t;u3(theta/2,phi,0) t;}
""";

In [93]:
function genQFT(qReg::String, qubitIndices::Vector{Int})
    circuit = ""
    angle(k) = 2*pi / 2^k;
    for (iidx,ival) = Iterators.reverse(enumerate(qubitIndices))
        circuit *= applyGateH(qReg, iidx)
        for (jidx,jval) = Iterators.reverse(enumerate(qubitIndices[1:iidx-1]))
            theta = angle(iidx - jidx);
            circuit *= applyGateCPhase(qReg, theta, jval, ival)
        end
    end
    for idx = 1:div(length(qubitIndices),2)
        circuit *= applySwap(qReg, v[idx], v[length(v)-idx+1])
    end
    return circuit
end

function genIQFT(qReg::String, qubitIndices::Vector{Int})
    circuit = ""
    angle(k) = -2*pi / 2^k;
    for (iidx,ival) = enumerate(qubitIndices)
        for (jidx,jval) = enumerate(qubitIndices[1:iidx-1])
            theta = angle(iidx - jidx);
            circuit *= applyGateCPhase(qReg, theta, jval, ival)
        end
        circuit *= applyGateH(qReg, iidx)

    end
    for idx = 1:div(length(qubitIndices),2)
        circuit *= applySwap(qReg, v[idx], v[length(v)-idx+1])
    end
    return circuit
end

genIQFT (generic function with 1 method)

In [99]:
cct = genQFT("q",v)

"U(1.5707963267948966,0,pi) q[4];U(0,0,1.5707963267948966) q[3];CX q[3],q[4];U(0,0,-1.5707963267948966) q[4];CX q[3],q[4];U(0,0,1.5707963267948966) q[4];U(0,0,0.7853981633974483) q[2];CX q[2],q[4];U(0,0,-0.7853981633974483) q[4];CX q[2],q[4];U(0,0,0.7853981633974483) q[4];U(0,0,0.39269908169872414) q[1];CX q[1],q[4];U(0,0,-0.39269908169872414) q[4];CX q[1],q[4];U(0,0,0.39269908169872414) q[4];U(1.5707963267948966,0,pi) q[3];U(0,0,1.5707963267948966) q[2];CX q[2],q[3];U(0,0,-1.5707963267948966) q[3];CX q[2],q[3];U(0,0,1.5707963267948966) q[3];U(0,0,0.7853981633974483) q[1];CX q[1],q[3];U(0,0,-0.7853981633974483) q[3];CX q[1],q[3];U(0,0,0.7853981633974483) q[3];U(1.5707963267948966,0,pi) q[2];U(0,0,1.5707963267948966) q[1];CX q[1],q[2];U(0,0,-1.5707963267948966) q[2];CX q[1],q[2];U(0,0,1.5707963267948966) q[2];U(1.5707963267948966,0,pi) q[1];CX q[1],q[4];CX q[4],q[1];CX q[1],q[4];CX q[2],q[3];CX q[3],q[2];CX q[2],q[3];"

In [100]:
function formatStringNL(circuitOutput::String)
    return replace(circuitOutput, ";"=>";\n")
end

formatStringNL (generic function with 1 method)

In [101]:
println(formatStringNL(cct))

U(1.5707963267948966,0,pi) q[4];
U(0,0,1.5707963267948966) q[3];
CX q[3],q[4];
U(0,0,-1.5707963267948966) q[4];
CX q[3],q[4];
U(0,0,1.5707963267948966) q[4];
U(0,0,0.7853981633974483) q[2];
CX q[2],q[4];
U(0,0,-0.7853981633974483) q[4];
CX q[2],q[4];
U(0,0,0.7853981633974483) q[4];
U(0,0,0.39269908169872414) q[1];
CX q[1],q[4];
U(0,0,-0.39269908169872414) q[4];
CX q[1],q[4];
U(0,0,0.39269908169872414) q[4];
U(1.5707963267948966,0,pi) q[3];
U(0,0,1.5707963267948966) q[2];
CX q[2],q[3];
U(0,0,-1.5707963267948966) q[3];
CX q[2],q[3];
U(0,0,1.5707963267948966) q[3];
U(0,0,0.7853981633974483) q[1];
CX q[1],q[3];
U(0,0,-0.7853981633974483) q[3];
CX q[1],q[3];
U(0,0,0.7853981633974483) q[3];
U(1.5707963267948966,0,pi) q[2];
U(0,0,1.5707963267948966) q[1];
CX q[1],q[2];
U(0,0,-1.5707963267948966) q[2];
CX q[1],q[2];
U(0,0,1.5707963267948966) q[2];
U(1.5707963267948966,0,pi) q[1];
CX q[1],q[4];
CX q[4],q[1];
CX q[1],q[4];
CX q[2],q[3];
CX q[3],q[2];
CX q[2],q[3];

